In [ ]:
import warnings
warnings.simplefilter("ignore", FutureWarning)
import json
import pandas as pd

from utils.config import Config
from utils.visualization import plot_overfitting, plot_test_metrics

### Check Metrics

In [ ]:
plot_overfitting()

In [ ]:
plot_test_metrics()

### Check Recommendation

In [ ]:
rec = json.load(open("../results/recommendation.json", "r"))
print("Ratings:", rec["TopModelRatings"], "\n")
for k, v in rec["MedianTopModelTestMetrics"].items():
    print(f"{k}: {round(v, 2)}")

In [ ]:
long = pd.DataFrame({k: v for k, v in rec["long"].items() if k in ["risk", "reward"]}).transpose()
short = pd.DataFrame({k: v for k, v in rec["short"].items() if k in ["risk", "reward"]}).transpose()
df = pd.concat([
    pd.concat({"long": long}, names=["PositionType"]),
    pd.concat({"short": short}, names=["PositionType"])
])
df

### Trade

In [ ]:
# Select
position_type = "short"
optimize = "risk"

trade = df.loc[position_type].loc[optimize]
print(trade["TopStock"])
trade["ISINDescription"] = Config().securities[trade["TopStock"]]
trade["PositionType"] = position_type
trade["Optimize"] = optimize
trade["TopModelRatings"] = rec["TopModelRatings"]
trade["MedianTopModelTestMetrics"] = rec["MedianTopModelTestMetrics"]
trade

In [ ]:
# Open
trade["WKN"] = ""
trade["WKN Description"] = ""
trade["Security Type"] = "turbo"
trade["Fundamentals"] = ""
trade["Git Tag"] = "20231102"
trade["Open Date"] = ""
trade["Open Price"] = 0
trade["Stop Limit"] = 0
trade["Shares"] = 0
trade

In [ ]:
# Close
trade["Close Price"] = 0
trade["Gross Profit"] = 0
trade["Fees"] = 0
trade["Net Profit"] = 0
trade["Reward"] = 0
trade

In [ ]:
# Persist
path = "../results/trades.csv"
trades = pd.read_csv(path, sep=";")
# pd.concat([trades, trade]).to_csv(path, sep=";", index=False)